## Import

In [ ]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import geoparquet as gpq
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg



from pathlib import Path
import os
import glob

import re
from tqdm import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Data

In [ ]:
# sa
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa, crs="EPSG:2193")

# pga 
pga = pd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pgaGeo = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193"))

In [ ]:
data = gpd.sjoin(pga, sa, op="within")

## Ignition calculations

In [ ]:
data['SA_IgnPrb'] = 1/(np.exp(-1*(-6.755+8.463+data[percentile]+0.0000984+data['PopDensity']+0.001523+data['BldgFloo_1'])))
data['IgnProb_bl'] = 1 - np.power(1-data['SA_IgnPrb'], +1/data['NumberBldg'])